# Self Drving: Vehicle Detection

### Enviroment Setup

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import random
import itertools
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 5)

from skimage.feature import hog

from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


random.seed(42)
VEHICLE_DIR = "./train/vehicles/"
NON_VEHICLE_DIR = "./train/non-vehicles/"

### Helper Functions 

In [ ]:
def read_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Define a function to compute binned color features  
def bin_spatial(image, color_space='RGB', size=(32, 32)):
    # Convert image to new color space (if specified)
    color_space_dict = {"HSV": cv2.COLOR_RGB2HSV, "HLS": cv2.COLOR_RGB2HLS, "YCrCb" :cv2.COLOR_RGB2YCrCb}
    
    img = cv2.resize(image, size)
    if color_space in color_space_dict.keys():
        image = cv2.cvtColor(image, color_space_dict[color_space])
    
    # Use cv2.resize().ravel() to create the feature vector
    features = image.ravel() # Remove this line!
    # Return the feature vector
    return features

def extract_features(image_path_list, color_channel, orientations, pix_per_cell, cell_per_block):
    COLOR_CHANNEL_DICT = {"HLS:S": (cv2.COLOR_RGB2HLS, 2), "YCrCb:Y": (cv2.COLOR_RGB2YCrCb, 0), "YCrCb:Cr": (cv2.COLOR_RGB2YCrCb, 1)}
    
    features = []
    
    for image_path in image_path_list:
        image = read_image(image_path)
    
        if  color_channel in COLOR_CHANNEL_DICT.keys():
            image = cv2.cvtColor(image, COLOR_CHANNEL_DICT[color_channel][0])[:,:,COLOR_CHANNEL_DICT[color_channel][1]]
        else:
            image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
        hog_features = get_hog_features(image, orientations, pix_per_cell, cell_per_block,  vis=False, feature_vec=True)
        features.append(hog_features)
    
    return features
    
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy
    

### Resource Loading 

In [ ]:
vehicle_image_path_list = glob.glob(VEHICLE_DIR + "./*/*.png")
non_vehicle_image_path_list = glob.glob(NON_VEHICLE_DIR + "./*/*.png")

print("vehicle image count: " + str(len(vehicle_image_path_list)) + " non vehicle image count: " + str(len(non_vehicle_image_path_list)))

#example output
for i in range(3):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3))
    f.tight_layout()
    vehicle_image = read_image(random.choice(vehicle_image_path_list))
    non_vehicle_image = read_image(random.choice(non_vehicle_image_path_list))
    ax1.imshow(vehicle_image)
    ax1.set_title("vehicle " + str(vehicle_image.shape))
    ax2.imshow(non_vehicle_image)
    ax2.set_title("non vehicle " + str(non_vehicle_image.shape))

plt.show()

### Histogram of Oriented Gradients (HOG)

#### Extracted HOG Features from the Training Images

In [ ]:
COLOR_CHANNEL_DICT = {"HLS:S": (cv2.COLOR_RGB2HLS, 2), "YCrCb:Y": (cv2.COLOR_RGB2YCrCb, 0), "YCrCb:Cr": (cv2.COLOR_RGB2YCrCb, 1)}

orientations = [8, 9]
pixels_per_cell = [6, 8]
cells_per_block = [2, 3]

parameter_combination_list = list(itertools.product(orientations, pixels_per_cell, cells_per_block))

car_image_path = random.choice(vehicle_image_path_list)
not_car_image_path = random.choice(non_vehicle_image_path_list)

for color_channel in COLOR_CHANNEL_DICT.keys():
    print(color_channel)
    car_image = read_image(car_image_path)
    not_car_image = read_image(not_car_image_path)
    
    color_space = COLOR_CHANNEL_DICT[color_channel][0]
    color_channel_index = COLOR_CHANNEL_DICT[color_channel][1] 
    
    car_image = cv2.cvtColor(car_image,color_space)[:,:,color_channel_index]
    not_car_image = cv2.cvtColor(not_car_image, color_space)[:,:,color_channel_index]
    
    for parameter_combination in parameter_combination_list: 
        car_features, car_hog_image = get_hog_features(car_image, parameter_combination[0], parameter_combination[1], parameter_combination[2], 
                        vis=True, feature_vec=True)
        
        not_car_features, not_car_hog_image = get_hog_features(not_car_image, parameter_combination[0], parameter_combination[1], parameter_combination[2], 
                        vis=True, feature_vec=True)
        
        f, axes = plt.subplots(1, 4, figsize=(16, 8))
        
        axes = axes.ravel()
        axes[0].imshow(car_image,cmap='gray')
        axes[0].set_title("car in " + color_channel)
        
        axes[1].imshow(car_hog_image,cmap='gray')
        axes[1].set_title("car HOG \n (orientations,pixels_per_cell, cells_per_block) = \n" + str(parameter_combination))
        
        axes[2].imshow(not_car_image, cmap='gray')
        axes[2].set_title("not car in " + color_channel)
        
        axes[3].imshow(not_car_hog_image,cmap='gray')
        axes[3].set_title("not car HOG\n (orientations,pixels_per_cell, cells_per_block) = \n" + str(parameter_combination))
        
        plt.show()
        

### SVM Classfier Trainning

#### Data Preparation

In [ ]:
vehicle_features = extract_features(vehicle_image_path_list, "YCrCb:Y", orientations = 8, pix_per_cell = 8, cell_per_block = 2)
non_vehicle_features = extract_features(non_vehicle_image_path_list, "YCrCb:Y", orientations = 8, pix_per_cell = 8, cell_per_block = 2)

print("vehicle image count: " + str(len(vehicle_features)) + " , non vehicle image count: " + str(len(non_vehicle_features)))

#### Build Classifier

In [ ]:
raw_X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64)
scaled_X = StandardScaler().fit_transform(raw_X)
print("X shape: " + str(scaled_X.shape))
y = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features))))
print("y shape: " + str(y.shape))

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=42)

parameter_dict = {'kernel':('linear','rbf'), 'C':[0.1,1,10]}
svr = SVC()
clf = GridSearchCV(svr, parameter_dict)
clf.fit(X_train, y_train)

print("Best Fit Parameter: ", clf.best_estimator_)



In [ ]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("accuracy = {:.4f}.".format(accuracy))